In [1]:
import gradio as gr
import numpy as np
import pandas as pd


def calculate_var(risk_min, risk_max, gain_min, gain_max, cost_of_risk, confidence_level):
    n_simulations = 5000
    np.random.seed(42)  # pour la reproductibilité

    # Calcul des paramètres pour la distribution des risques
    risk_mean = (risk_min + risk_max) / 2 / 100
    risk_stddev = (risk_max - risk_min) / 6 / 100

    # Calcul des paramètres pour la distribution des gains
    gain_mean = (gain_min + gain_max) / 2
    gain_stddev = (gain_max - gain_min) / 6

    # Simuler les pertes en millions de CAD
    losses = cost_of_risk * np.random.normal(risk_mean, risk_stddev, n_simulations)
    # Simuler les gains en millions de CAD
    gains = np.random.normal(gain_mean, gain_stddev, n_simulations)

    # Calcul de la valeur nette (gain - perte)
    net_values = gains - losses

    # Calculer la VaR de la valeur nette pour le quantile (1 - confidence level)
    var_percentile = np.percentile(net_values, (1 - confidence_level) * 100)
    mean_net_value = np.mean(net_values)
    std_net_value = np.std(net_values)

    # Calcul de l'histogramme en tant que DataFrame
    counts, bins = np.histogram(net_values, bins=50, density=True)
    bin_centers = (bins[:-1] + bins[1:]) / 2  # Centres des bins

    # Conversion de la densité en pourcentage
    counts_percentage = counts * 100

    # Créer un DataFrame pour Gradio
    df = pd.DataFrame({"value": bin_centers, "probability": counts_percentage})


    # Rapport des résultats en millions de CAD
    report = f"""
    ===== Valeur nette du projet en millions de CAD =====
    Coût du risque initial : {cost_of_risk} millions CAD
    Niveau de confiance : {confidence_level * 100:.0f}%
    VaR estimée de la valeur nette : {var_percentile:.2f} millions CAD
    Valeur nette moyenne simulée : {mean_net_value:.2f} millions CAD
    Écart-type des valeurs nettes simulées : {std_net_value:.2f} millions CAD
    =====================================================
    """
    return df, report

# Interface Gradio
iface = gr.Interface(
    fn=calculate_var,
    inputs=[
        gr.Number(label="Risque minimum (%)", value=5),
        gr.Number(label="Risque maximum (%)", value=20),
        gr.Number(label="Gain minimum (millions CAD)", value=10),
        gr.Number(label="Gain maximum (millions CAD)", value=20),
        gr.Number(label="Coût du risque (millions CAD)", value=70),
        gr.Slider(0.80, 0.99, step=0.01, label="Niveau de confiance", value=0.95),
    ],
    outputs=[
        #gr.Image(type="filepath", label="Distribution des valeurs nettes simulées"),
        gr.BarPlot(x="value", y="probability", x_bin=1, y_aggregate="mean", title="Distribution des valeurs nettes simulées (en millions de CAD)", x_label="Valeur nette (en millions de CAD)", y_label="Densité de probabilité (%)"),
        gr.Textbox(label="Rapport de Valeur Nette à Risque (VNaR)")
    ],
    title="Calcul de la Valeur Nette à Risque (VNaR) pour un projet",
    description="Entrez les paramètres pour estimer la Valeur Nette à Risque (VNaR) d'un projet."
)

iface.launch(inline=False, share=True)


C:\Users\delem\VNaR\monenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://60d3be768aa213e502.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
